In [1]:
import os
import itertools
import tensorflow as tf
import early_stopping
from keras import backend as K
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import tools_with_GMP as tools
import plot_confusion_matrix_Copy1 as plot_confusion_matrix
from sklearn.metrics import confusion_matrix
import numpy as np
np.set_printoptions(suppress=True)

training_data, training_label, validation_data, validation_label, validation_cate_label = tools.get_data()


/home/hsiehch/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/hsiehch/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/hsiehch/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/hsiehch/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:5

data: 8528
label: 8528
StratifiedKFold(n_splits=5, random_state=100, shuffle=True)
trian: [   0    1    2 ... 8525 8526 8527] len 6821 test: [  10   12   19 ... 8515 8520 8524] len 1707
trian: [   0    1    2 ... 8524 8525 8527] len 6822 test: [   3    9   15 ... 8509 8521 8526] len 1706
trian: [   0    2    3 ... 8525 8526 8527] len 6822 test: [   1    5    6 ... 8513 8519 8523] len 1706
trian: [   0    1    2 ... 8524 8526 8527] len 6823 test: [   8   11   26 ... 8517 8518 8525] len 1705
trian: [   1    3    5 ... 8524 8525 8526] len 6824 test: [   0    2    4 ... 8512 8522 8527] len 1704


In [28]:
sum(training_label[0])

array([ 606.,  223., 4060., 1932.], dtype=float32)

In [2]:
def foo(training_data, training_label):
    n = 900
    ii = 1
    trainX = []
    trainY = []
    for dat, lab in zip(training_data, training_label):

        if ii <= 900 and (np.argmax(lab) == 2):
            trainX.append(dat)
            trainY.append(lab)
            ii = ii + 1
        if np.argmax(lab) != 2:
            trainX.append(dat)
            trainY.append(lab)
    trainX = np.array(trainX)
    trainY = np.array(trainY)
    return trainX, trainY
    

In [2]:
from keras_radam.optimizer_v2 import RAdam
ks = 3
num_layer = 5
bs = 30
lr = 0.0001
epochs = 100
scores = []

def create_model(learning_rate, bs, ks, num_layer):
    num_filter = 32
    
    in_data = tf.keras.Input(shape=(None, 1), dtype="float64")
    x = tf.keras.layers.Conv1D(filters = num_filter, kernel_size = ks, activation="relu")(in_data)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPool1D(2)(x)

    for i in range(2,num_layer+1):
        try:
            if i==num_layer:
                x = tf.keras.layers.Conv1D(filters = num_filter, kernel_size = ks, activation="relu")(x)
                break
            if i%2 != 0:
                num_filter = num_filter *2
            x = tf.keras.layers.Conv1D(filters = num_filter, kernel_size = ks, activation="relu")(x)
            x = tf.keras.layers.MaxPool1D(2)(x)
            if i in [6, 8, 9]:
                x = tf.keras.layers.Dropout(0.5)(x)
        except ValueError:
            print("model overflow[lr, bs, ks, #layer]: ",[learning_rate, bs, ks, i+1])
            return False
    
    x = tf.keras.layers.GlobalMaxPool1D()(x)
    x = tf.keras.layers.Dense(64, activation = 'relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(32, activation = 'relu')(x)
    outputs = tf.keras.layers.Dense(4, activation = 'softmax')(x)
    
    model = tf.keras.Model(inputs=in_data, outputs=outputs)
    print(model.summary())
    return model

def run(bs, lr, ks, num_layer):
    fold=1
    for index, (X_train, Y_train, X_val, Y_val, val_cat) in enumerate(zip(training_data,
                                                       training_label,
                                                       validation_data,
                                                       validation_label,
                                                       validation_cate_label)):

                
        X_val, Y_val, val_cat = shuffle(X_val, Y_val, val_cat, random_state=50)
        ES = early_stopping.EarlyStopping(patience=6)
        model = create_model(lr, bs, ks, num_layer)
        optimizer = tf.keras.optimizers.Adam(lr = lr)
#         optimizer = tf.keras.optimizers.SGD()
#         optimizer = RAdam(learning_rate=lr)
        
        losses = []
        
        for epoch in range(1, epochs+1, 1):
            print("Epoch: {}".format(epoch))
            epoch_losses = []
            
            prog_bar = tf.keras.utils.Progbar(X_train.shape[0])
            prog_bar_val = tf.keras.utils.Progbar(X_val.shape[0])
            train_acc_metric = tf.keras.metrics.Accuracy()
            val_acc_metric = tf.keras.metrics.Accuracy()
            
            X_train, Y_train = shuffle(X_train, Y_train, random_state=0)
            
            for ind, (input_data, input_label) in enumerate(zip(X_train, Y_train)):
                
                with tf.GradientTape() as tape:
                    input_data = input_data.reshape((1, input_data.shape[0], 1))
                    input_label = input_label.reshape((1, input_label.shape[0]))
                    logits = model(input_data)
                    loss_value = tf.keras.losses.categorical_crossentropy(input_label, logits)
                    epoch_losses.append(float(loss_value))
                
                train_acc_metric.update_state(np.argmax(input_label), np.argmax(logits))
                prog_bar.add(1, values=[("train loss", float(loss_value)), ("train accuracy", float(train_acc_metric.result()))])
                
                # update weights using mini-batch mechanism
                if (ind+1)%bs == 0:
                    grads = tape.gradient(loss_value, model.trainable_weights)
                    optimizer.apply_gradients(zip(grads, model.trainable_weights))
            
            
            # weights update for the last batch
            grads = tape.gradient(loss_value, model.trainable_weights)
            optimizer.apply_gradients(zip(grads, model.trainable_weights))
            
            avg_epoch_loss = sum(epoch_losses) / (1.0 * len(epoch_losses))
            print("{}: {}".format(epoch, avg_epoch_loss))
            losses.append(avg_epoch_loss)
            
            for ind, (input_data, input_label) in enumerate(zip(X_val, Y_val)):
                input_data = input_data.reshape((1, input_data.shape[0], 1))
                input_label = input_label.reshape((1, input_label.shape[0]))
                val_logits = model(input_data)
                val_loss_val = tf.keras.losses.categorical_crossentropy(input_label, val_logits)
                val_acc_metric.update_state(np.argmax(input_label), np.argmax(logits))
                prog_bar_val.add(1, values=[("val loss", float(val_loss_val)), ("val accuracy", float(val_acc_metric.result()))])
        
            print("\n")
            
            ES(float(val_loss_val), model, fold)
            if ES.early_stop:
                print("Early Stopping!")
                break
        
        val_pred_cat = []
        for ind, (input_data, input_label) in enumerate(zip(X_val, Y_val)):
            input_data = input_data.reshape((1, input_data.shape[0], 1))
            input_label = input_label.reshape((1, input_label.shape[0]))
            val_pred = model.predict(input_data)
            val_pred_cat.append(np.argmax(val_pred))
        
        score = f1_score(val_cat, val_pred_cat, average=None)
        scores.append(score)
        print(score)
        
        fold = fold + 1
        
#         cnf_matrix = confusion_matrix(val_cat, val_pred_cat)
#         plot_confusion_matrix.plot_confusion_matrix(cnf_matrix, classes=['AF','Noise','Normal','Other'], save_png=True)
        
        print("\n")

run(bs, lr, ks, num_layer)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, 1)]         0         
_________________________________________________________________
conv1d (Conv1D)              (None, None, 32)          128       
_________________________________________________________________
batch_normalization (BatchNo (None, None, 32)          128       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, None, 32)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 32)          3104      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, None, 32)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, None, 64)          6208  

W0827 09:50:11.179541 139706075281152 deprecation.py:323] From /home/hsiehch/.local/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1220: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


6821/6821 [==============================] - 165s 24ms/step - train loss: 1.2829 - train accuracy: 0.4979
1: 1.2829127131622065
1707/1707 [==============================] - 199s 117ms/step - val loss: 1.1359 - val accuracy: 0.5958


Epoch: 2
6821/6821 [==============================] - 166s 24ms/step - train loss: 1.0983 - train accuracy: 0.5919
2: 1.0982760400894422
1707/1707 [==============================] - 201s 118ms/step - val loss: 1.0910 - val accuracy: 0.5958


Epoch: 3
6821/6821 [==============================] - 164s 24ms/step - train loss: 1.0631 - train accuracy: 0.5987
3: 1.0631106566781139
1707/1707 [==============================] - 198s 116ms/step - val loss: 1.0559 - val accuracy: 0.5958


Epoch: 4
6821/6821 [==============================] - 165s 24ms/step - train loss: 1.0506 - train accuracy: 0.5949
4: 1.050601203740609
1707/1707 [==============================] - 199s 116ms/step - val loss: 1.0453 - val accuracy: 0.5958


Epoch: 5
6821/6821 [======================

/opt/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[0.         0.         0.74623577 0.        ]


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None, 1)]         0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, None, 32)          128       
_________________________________________________________________
batch_normalization_1 (Batch (None, None, 32)          128       
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, None, 32)          0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, None, 32)          3104      
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, None, 32)          0         
_________________________________________________________________
conv1d_7 (C

...

[0.         0.         0.74604925 0.        ]


Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, None, 1)]         0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, None, 32)          128       
_________________________________________________________________
batch_normalization_2 (Batch (None, None, 32)          128       
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, None, 32)          0         
_________________________________________________________________
conv1d_11 (Conv1D)           (None, None, 32)          3104      
_________________________________________________________________
max_pooling1d_9 (MaxPooling1 (None, None, 32)          0         
_________________________________________________________________
conv1d_12 (

6822/6822 [==============================] - 179s 26ms/step - train loss: 0.9753 - train accuracy: 0.6078
24: 0.9752744861616395
1706/1706 [==============================] - 217s 127ms/step - val loss: 0.9852 - val accuracy: 0.6094


Early Stopping!


/opt/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[0.         0.         0.74604925 0.        ]


Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, None, 1)]         0         
_________________________________________________________________
conv1d_15 (Conv1D)           (None, None, 32)          128       
_________________________________________________________________
batch_normalization_3 (Batch (None, None, 32)          128       
_________________________________________________________________
max_pooling1d_12 (MaxPooling (None, None, 32)          0         
_________________________________________________________________
conv1d_16 (Conv1D)           (None, None, 32)          3104      
_________________________________________________________________
max_pooling1d_13 (MaxPooling (None, None, 32)          0         
_________________________________________________________________
conv1d_17 (

KeyboardInterrupt: 